In [207]:
import pandas as pd
import numpy as np

In [208]:
scouts_old = pd.read_csv("./2017_scouts.csv")

In [209]:
scouts_old.columns

Index(['Rodada', 'ClubeID', 'AtletaID', 'Participou', 'Pontos', 'PontosMedia',
       'Preco', 'PrecoVariacao', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I',
       'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS'],
      dtype='object')

In [211]:
scouts_old.head(10)

,Rodada,ClubeID,AtletaID,Participou,Pontos,PontosMedia,Preco,PrecoVariacao,FS,PE,...,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,0,262,36540,False,0.0,0.00,5.00,0.00,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,262,36540,False,0.0,0.00,5.00,0.00,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,262,36540,False,0.0,0.00,5.00,0.00,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,262,36540,False,0.0,0.00,5.00,0.00,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,262,36540,True,7.5,7.50,6.11,1.11,0,1,...,0,2,2,0,1,0,1,0,0,0
5,5,262,36540,True,2.6,5.05,5.64,-0.47,1,2,...,0,4,2,0,2,0,1,0,0,0
6,6,262,36540,True,10.4,6.83,7.76,2.12,2,5,...,0,7,2,0,2,0,1,0,0,0
7,8,262,36540,True,0.2,5.17,6.25,-1.51,4,6,...,0,7,3,0,2,0,1,0,0,0
8,9,262,36540,True,-0.8,3.98,5.92,-0.33,4,6,...,0,8,4,0,3,0,1,0,0,0
9,10,262,36540,False,0.0,3.98,5.92,0.00,4,6,...,0,8,4,0,3,0,1,0,0,0


In [212]:
scouts_old = scouts_old[['Rodada', 'ClubeID', 'AtletaID', 'Participou', 'Pontos', 'PontosMedia', 'PrecoVariacao', 'Preco', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS']]

In [213]:
def disaggregator(db, player_id):
    player_scouts = db.query("AtletaID == @player_id").sort_values("Rodada", ascending = False)
    subtract = player_scouts.iloc[1:player_scouts.shape[0]]
    sub = player_scouts.iloc[player_scouts.shape[0]-1]
    sub.iloc[8:26] = 0
    subtract = subtract.append(sub, ignore_index = True)
    ans = player_scouts
    ans.index = subtract.index
    price = ans.Preco.copy()
    ans.iloc[:,7:26] = ans.iloc[:,7:26] - subtract.iloc[:,7:26]
    ans["price"] = price
    return ans

In [214]:
scouts_disagg = pd.DataFrame(columns = list(scouts_old.columns) + ["price"])
for player_id in scouts_old.AtletaID.unique():
    scouts_disagg = scouts_disagg.append(disaggregator(scouts_old, player_id), ignore_index=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [215]:
scouts_filtered = scouts_disagg.copy()
scouts_filtered.PrecoVariacao = np.round(scouts_filtered.PrecoVariacao,2)
scouts_filtered.Preco = np.round(scouts_filtered.Preco,2)

In [220]:
scouts_filtered.query("PrecoVariacao != Preco")

,Rodada,ClubeID,AtletaID,Participou,Pontos,PontosMedia,PrecoVariacao,Preco,FS,PE,...,RB,FC,GC,CA,CV,SG,DD,DP,GS,price
68,8,275,36612,False,0.00,1.77,0.00,-0.47,2,3,...,0,1,0,0,0,0,0,0,0,7.28
129,8,283,37281,True,6.85,3.82,1.02,0.30,0,0,...,0,0,0,0,0,0,0,0,0,12.59
146,29,315,37306,True,0.00,3.76,0.00,-0.33,0,0,...,0,0,0,0,0,0,0,0,0,12.04
162,8,303,37306,True,4.39,4.25,0.55,-0.07,0,0,...,0,0,0,0,0,0,0,0,0,12.18
438,8,314,37646,False,0.00,1.85,0.00,-0.14,2,2,...,2,2,0,1,0,0,0,0,0,5.99
514,8,282,37655,True,9.80,3.30,1.39,2.40,4,4,...,2,7,0,0,0,0,0,0,0,7.08
552,8,283,37656,True,6.00,4.57,0.55,0.50,0,0,...,0,0,0,0,0,0,6,0,4,13.26
590,8,275,37657,False,0.00,4.55,0.00,-0.78,2,2,...,1,1,0,0,0,0,0,0,0,9.23
628,8,293,37662,True,7.50,2.74,0.99,1.88,1,3,...,3,2,0,1,0,2,0,0,0,7.93
676,8,294,37673,True,-0.10,-0.32,-0.07,-0.69,1,5,...,1,2,0,0,0,0,0,0,0,3.55


In [217]:
scouts_filtered.Rodada.unique()

array([38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22,
       21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 6, 5, 4, 3,
       2, 1, 0], dtype=object)

In [219]:
scouts_filtered.query("PrecoVariacao != Preco").Rodada.unique()

array([8, 29, 12, 3, 17, 9, 24, 10, 4, 1, 15, 20, 25, 22, 2, 14, 16, 38],
      dtype=object)

In [232]:
scouts_old.query("AtletaID == 97448")

,Rodada,ClubeID,AtletaID,Participou,Pontos,PontosMedia,PrecoVariacao,Preco,FS,PE,...,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
27718,14,294,97448,True,-0.8,-0.80,-0.80,1.20,0,1,...,0,0,0,0,0,0,0,0,0,0
27719,15,294,97448,False,0.0,-0.80,0.00,1.20,0,1,...,0,0,0,0,0,0,0,0,0,0
27720,16,294,97448,False,0.0,-0.80,0.00,1.20,0,1,...,0,0,0,0,0,0,0,0,0,0
27721,17,294,97448,False,0.0,-0.80,0.00,1.20,0,1,...,0,0,0,0,0,0,0,0,0,0
27722,18,294,97448,True,0.0,-0.40,-0.21,0.99,0,1,...,0,0,0,0,0,0,0,0,0,0
27723,19,294,97448,False,0.0,-0.40,0.00,0.99,0,1,...,0,0,0,0,0,0,0,0,0,0
27724,20,294,97448,False,0.0,-0.40,0.00,0.99,0,1,...,0,0,0,0,0,0,0,0,0,0
27725,21,294,97448,False,0.0,-0.40,0.00,0.99,0,1,...,0,0,0,0,0,0,0,0,0,0
27726,22,294,97448,False,0.0,-0.40,0.00,0.99,0,1,...,0,0,0,0,0,0,0,0,0,0
27727,23,294,97448,False,0.0,-0.40,0.00,0.99,0,1,...,0,0,0,0,0,0,0,0,0,0


In [230]:
scouts_filtered.query("PrecoVariacao != Preco")

,Rodada,ClubeID,AtletaID,Participou,Pontos,PontosMedia,PrecoVariacao,Preco,FS,PE,...,RB,FC,GC,CA,CV,SG,DD,DP,GS,price
68,8,275,36612,False,0.00,1.77,0.00,-0.47,2,3,...,0,1,0,0,0,0,0,0,0,7.28
129,8,283,37281,True,6.85,3.82,1.02,0.30,0,0,...,0,0,0,0,0,0,0,0,0,12.59
146,29,315,37306,True,0.00,3.76,0.00,-0.33,0,0,...,0,0,0,0,0,0,0,0,0,12.04
162,8,303,37306,True,4.39,4.25,0.55,-0.07,0,0,...,0,0,0,0,0,0,0,0,0,12.18
438,8,314,37646,False,0.00,1.85,0.00,-0.14,2,2,...,2,2,0,1,0,0,0,0,0,5.99
514,8,282,37655,True,9.80,3.30,1.39,2.40,4,4,...,2,7,0,0,0,0,0,0,0,7.08
552,8,283,37656,True,6.00,4.57,0.55,0.50,0,0,...,0,0,0,0,0,0,6,0,4,13.26
590,8,275,37657,False,0.00,4.55,0.00,-0.78,2,2,...,1,1,0,0,0,0,0,0,0,9.23
628,8,293,37662,True,7.50,2.74,0.99,1.88,1,3,...,3,2,0,1,0,2,0,0,0,7.93
676,8,294,37673,True,-0.10,-0.32,-0.07,-0.69,1,5,...,1,2,0,0,0,0,0,0,0,3.55


In [132]:
scouts_filtered.Participou.loc[scouts_filtered.query("PrecoVariacao != Preco and Rodada not in (27,37)").index] = True

In [191]:
scouts_filtered = scouts_old

In [233]:
scouts_filtered = scouts_filtered.query("Participou == True")

In [234]:
col_names = ["round_num", "player_id", "player_name", "player_nickname", "position_id", "team_id", "G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC", "total_score", "current_price"]

In [236]:
players = pd.read_csv("./2017_atletas.csv")

In [237]:
players.head()

,ID,Apelido,ClubeID,PosicaoID
0,36540,Juan,262,3
1,36612,Zé Roberto,275,2
2,36940,Oswaldo de Oliveira,282,6
3,36943,Paulo Autuori,293,6
4,37245,Guto Ferreira,265,6


In [238]:
players.Apelido.unique()

array(['Juan', 'Zé Roberto', 'Oswaldo de Oliveira', 'Paulo Autuori',
       'Guto Ferreira', 'Ney Franco', 'Mano Menezes', 'Gilson Kleina',
       'Marcelo Cabo', 'Pintado', 'Paulo André', 'Michel Bastos',
       'Cristian', 'Rafael Marques', 'Diguinho', 'Rodrigo',
       'Marcelo Mattos', 'Betão', 'Carlos Alberto', 'Nilmar',
       'Rafael Moura', 'Fábio', 'Edu Dracena', 'Jonathan', 'Julio Cesar',
       'Alecsandro', 'Wagner', 'Cícero', 'Henrique', 'Léo Moura',
       'Arouca', 'Marcelo Grohe', 'Edinho', 'Rafael Sobis', 'Gustavo',
       'Maicosuel', 'Luis Fabiano', 'Roger Bernardo', 'Aranha',
       'Anderson', 'Wendel', 'Lugano', 'Danilo', 'Durval', 'Jefferson',
       'Jô', 'Fernando Prass', 'Marquinhos', 'Artur Moraes', 'Fred',
       'Maicon', 'Muriqui', 'Fábio Santos', 'Grafite', 'Robinho',
       'Thiago Neves', 'Wellington Paulista', 'Roger', 'Márcio Araújo',
       'Jadson', 'Thiago Heleno', 'Vinícius Pacheco', 'Rafinha',
       'Fábio Ferreira', 'Diego Cavalieri', 'Pierre',

In [41]:
players.Apelido = players.Apelido.str.replace("Ã³", "ó")

In [239]:
scouts_filtered.AtletaID = scouts_filtered.AtletaID.apply(int)

In [240]:
scouts_filtered = scouts_filtered.merge(players, left_on = "AtletaID", right_on = "ID", how = "left")

In [241]:
scouts_filtered.query("ID != ID")

,Rodada,ClubeID_x,AtletaID,Participou,Pontos,PontosMedia,PrecoVariacao,Preco,FS,PE,...,CV,SG,DD,DP,GS,price,ID,Apelido,ClubeID_y,PosicaoID


In [242]:
scouts_filtered["player_name"] = None

In [243]:
scouts_filtered.columns

Index(['Rodada', 'ClubeID_x', 'AtletaID', 'Participou', 'Pontos',
       'PontosMedia', 'PrecoVariacao', 'Preco', 'FS', 'PE', 'A', 'FT', 'FD',
       'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP',
       'GS', 'price', 'ID', 'Apelido', 'ClubeID_y', 'PosicaoID',
       'player_name'],
      dtype='object')

In [244]:
scouts = scouts_filtered[['Rodada', 'AtletaID', 'player_name', 'Apelido', 'PosicaoID', 'ClubeID_x', "G", "A", "FT", "FD", "FF", "FS", "PP", "I", "PE", "SG", "DP", "DD", "RB", "GC", "CV", "CA", "GS", "FC", 'Pontos', 'Preco']]

In [245]:
scouts.columns = col_names

In [246]:
scouts

,round_num,player_id,player_name,player_nickname,position_id,team_id,G,A,FT,FD,...,DP,DD,RB,GC,CV,CA,GS,FC,total_score,current_price
0,35,36540,None,Juan,3,262,0,0,0,0,...,0,0,0,0,0,0,0,0,0.50,-1.44
1,31,36540,None,Juan,3,262,0,0,0,0,...,0,0,4,0,0,1,0,1,10.00,1.84
2,29,36540,None,Juan,3,262,0,0,0,0,...,0,0,1,0,0,1,0,3,-1.30,-0.81
3,27,36540,None,Juan,3,262,0,0,0,0,...,0,0,2,0,0,0,0,0,2.80,-0.49
4,22,36540,None,Juan,3,262,0,0,0,0,...,0,0,1,0,0,0,0,0,6.40,0.37
5,18,36540,None,Juan,3,262,0,0,0,0,...,0,0,2,0,0,0,0,1,4.10,-0.03
6,17,36540,None,Juan,3,262,0,1,1,1,...,0,0,1,0,0,0,0,1,5.90,0.39
7,16,36540,None,Juan,3,262,0,0,0,0,...,0,0,0,0,0,0,0,0,4.00,0.37
8,9,36540,None,Juan,3,262,0,0,0,0,...,0,0,1,0,0,1,0,1,-0.80,-0.33
9,8,36540,None,Juan,3,262,0,0,0,0,...,0,0,0,0,0,0,0,1,0.20,-1.51


In [247]:
for rnd in range(1,39):
    if rnd < 10:
        filename = "../2017/scouts/2017_round0" + str(rnd) + "_scouts.csv"
    else:
        filename = "../2017/scouts/2017_round" + str(rnd) + "_scouts.csv"
    scouts.query("round_num == @rnd").to_csv(filename, index = False)